In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
import pandas as pd
from skimage.metrics import structural_similarity

Now that we have folders with frames from each video for each channelId, we can start the analysis. There is two parts to this: 
1. Color analysis
2. Motion analysis

### Part I  <br/>
***Color Analysis*** <br/>
Is there a correlation between color and subcount?

In [ ]:
def getFeatureVectorOfImage_color (imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    for i, col in enumerate(['b', 'g', 'r']):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        vectors.append(hist)
        # plt.plot(hist, color = col)
        # plt.xlim([0, 256])
    # plt.show()
    vectors = np.vstack(vectors)
    return vectors

def getFeatureVectorOfImage (imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    vectors = np.vstack(hist)
    return vectors

def MSE_images (f1_v, f2_v, listOfMse): #small changes are even smaller, big changes are even bigger. 
    mse = np.sum(np.square(np.subtract(f1_v, f2_v))) / (256*256)
    listOfMse.append(mse)
    return mse

from IPython.display import Image, display
def non_whiteBlack_ratio(imagePath):
    img = cv2.imread(imagePath, 0)
    # display(Image(filename=imagePath) )
    print(img)
    number_of_white_pix = np.sum(img == 255)
    number_of_black_pix = np.sum(img == 0)
    number_of_pix = np.sum(img != None)
    print(number_of_white_pix, number_of_black_pix, number_of_pix)
    colorPercentage = (256**2-number_of_black_pix-number_of_white_pix)/ number_of_pix
    return colorPercentage

# non_whiteBlack_ratio("frames/_uk_6vfqwTA/00032.png")
# non_whiteBlack_ratio("frames/_uk_6vfqwTA/00317.png")
# non_whiteBlack_ratio("frames/_uk_6vfqwTA/22317.png")
# non_whiteBlack_ratio("frames/_uk_6vfqwTA/23772.png")
# non_whiteBlack_ratio("purered.png")
non_whiteBlack_ratio("why_white.png")

vidId_to_colorPercentage = {}
videosToAnalyze = glob.glob("frames/*")

# for i in range(len(videosToAnalyze)):
#     frames = glob.glob(videosToAnalyze[i]+'/*')
#     ratio = non_whiteBlack_ratio(frames[0])
#     for j in range(1,len(frames)):
#         ratio = (ratio+ non_whiteBlack_ratio(frames[j]))/2
#     vidId_to_colorPercentage[videosToAnalyze[i].split('/')[-1]]= ratio

count = 0
for i in range(len(videosToAnalyze)):
    frames = glob.glob(videosToAnalyze[i]+'/*')
    count+=len(frames)
print(count)
print(count/43)

In [ ]:
# with open('vidId_to_colorPercentage.json', 'w') as output:
#     json.dump(vidId_to_colorPercentage_2, output)
with open('vidId_to_colorPercentage.json') as input:
    vidId_to_colorPercentage_2 = json.load(input)

vidId_to_colorPercentage_2

In [ ]:
# with open('vidIdToMseColor.json', 'w') as output:
#     json.dump(vidId_to_Mse, output)
with open('vidIdToMseColor.json') as input:
    data = json.load(input)
data

In [ ]:
from collections import Counter

def binAndPlot(some_list):
    sorted_list = sorted(some_list)
    sorted_counted = Counter(sorted_list)
    range_length = list(range(int(max(some_list)+1))) # Get the largest value to get the range.
    data_series = {}

    for i in range_length:
        data_series[i] = 0 # Initialize series so that we have a template and we just have to fill in the values.
    for key, value in sorted_counted.items():
        data_series[key] = value
    data_series = pd.Series(data_series)
    x_values = data_series.index
    plt.hist(some_list, edgecolor="yellow", color="green")
    plt.show()

### Part 2:
*** Motion Analysis ***

Is there a correlation between the amount of animation and subcount?

In [ ]:
with open('../channelId_to_mostViewedVidId.json', 'r') as j:
    channelId_to_mostViewedVidId = json.loads(j.read())

LIST_OF_CHANNEL_IDS= channelId_to_mostViewedVidId.keys()
print(len(LIST_OF_CHANNEL_IDS))
channelId_to_mostViewedVidId

In [ ]:
with open('../masterSheetOfChannelIdToListOfVidInfos.json', 'r') as j:
    masterSheet = json.loads(j.read())

channelId_to_mostViewedVidInfo = {}

for channelId in LIST_OF_CHANNEL_IDS:
    mostViewedVidId = channelId_to_mostViewedVidId[channelId]
    for vidInfo in masterSheet[channelId]:
        if vidInfo['vidId'] == mostViewedVidId:
            channelId_to_mostViewedVidInfo[channelId] = vidInfo
channelId_to_mostViewedVidInfo
# print(len(channelId_to_mostViewedVidId))

In [ ]:
import regex as re
channelId_to_mostViewedVidInfoDf = pd.DataFrame.from_dict(channelId_to_mostViewedVidInfo).T
not_candidate_vidIds= ['kQEtRoyFfI8', 'hpQQohcHk9Q', '6wS_uON5s6Q', 'RlU32AfEVeU', 'iVqhzEaJhDw', 'Xnv7JGqjaAo', '18msRdBF11A', 'Ln4AnsWNUQI', 'Hkz0NcKPzMs']

#https://en.wikipedia.org/wiki/ISO_8601#Durations
for channelId, row in channelId_to_mostViewedVidInfoDf.iterrows():
    x= row['duration']
    m=0
    s=0
    try:
        m, s = re.findall('PT(\d+)M(\d+)S',x)[0]
    except:
        s = re.findall('PT(\d+)S',x)[0]
    if int(m)*60+int(s) < 210:
        not_candidate_vidIds.append(row['vidId'])
not_candidate_vidIds = set(not_candidate_vidIds)
candidateVideos = list(set(channelId_to_mostViewedVidInfoDf.vidId)- not_candidate_vidIds)
channelId_to_mostViewedVidInfoDf = channelId_to_mostViewedVidInfoDf[channelId_to_mostViewedVidInfoDf.vidId.isin(candidateVideos)]
print(len(channelId_to_mostViewedVidInfoDf))
channelId_to_mostViewedVidInfoDf

couple that do not fit the criteria:: vidIds [kQEtRoyFfI8- gameplay, hpQQohcHk9Q- too short. not commentary, 6wS_uON5s6Q- speeddrawing, RlU32AfEVeU- footage apparently from another youtuber, iVqhzEaJhDw- too short, Xnv7JGqjaAo- speeddraw, iRBmUQQzpWQ- simply not animation, 18msRdBF11A- not animation, Ln4AnsWNUQI- not animation, Hkz0NcKPzMs- too short]



The plan is to find the distribution of DIFF. It'll vary based on the artist.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from IPython.display import Image, display
import multiprocessing
import functools
from multiprocessing import Process

dictOfVidIdToDiffMse = {}

def getFeatureVectorOfImage_color (imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    for i, col in enumerate(['b', 'g', 'r']):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        vectors.append(hist)
        # plt.plot(hist, color = col)
        # plt.xlim([0, 256])
    # plt.show()
    vectors = np.vstack(vectors)
    return vectors

def getFeatureVectorOfImage (imagePath):
    image = cv2.imread(imagePath)
    vectors = []
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    vectors = np.vstack(hist)
    return vectors

def MSE_images (f1_v, f2_v, listOfMSE): #small changes are even smaller, big changes are even bigger. 
    mse = np.sum(np.square(np.subtract(f1_v, f2_v)))/(256*256)
    listOfMSE.append(mse)
    return mse

def DIFF_images(f1_v, f2_v, listOfDIFF): #downside 3-1 vs 1-3 is the same. 
    listOfDIFF.append((np.abs(f1_v - f2_v)).mean())

def SSIM_images(image1, image2, listOfSSIM): #more structural. 
    (score, _) = structural_similarity(cv2.cvtColor(cv2.imread(image1), cv2.COLOR_BGR2GRAY), cv2.cvtColor(cv2.imread(image2), cv2.COLOR_BGR2GRAY), full=True)
    listOfSSIM.append(score)
    return score

def smap(f, *args):
    return f(*args)

def smap(f):
    return f()

def getMeanMedianStdVar(lst):
        return {"mean": np.mean(np.array(lst)), "med": np.median(np.array(lst)), "std": np.std(np.array(lst)), "var": np.var(np.array(lst))}
            
#https://ece.uwaterloo.ca/~z70wang/publications/ssim.pdf
def removeLowDiff_andDemo(folderName):
    vidId = folderName.split('/')[-1]
    frameList = glob.glob(folderName+'/*.png') #unfortunately, doesn't work bc sortByInt isn't a thing
    validFiles = 0 
    validFrameList = []

    lowRangeDone= False
    midRangeDone= False
    highRangeDone= False

    listOfMSE = []
    listOfSIMM=[]

    imageShape = cv2.imread(frameList[0]).shape
    for i in range(len(frameList)-1):
        firstFrame = '{}/{}.png'.format(folderName, str(i).zfill(5))
        secondFrame = '{}/{}.png'.format(folderName, str(i+1).zfill(5))
        f1_v = getFeatureVectorOfImage_color(firstFrame)
        f2_v = getFeatureVectorOfImage_color(secondFrame)
        mse = MSE_images(f1_v, f2_v, listOfMSE)
        score = SSIM_images(firstFrame, secondFrame, listOfSIMM)
        listOfMSE.append(mse)
        listOfSIMM.append(score)
        if (score < 0.94):
            validFrameList.append(firstFrame)
            validFiles+=1

    dictOfVidIdToDiffMse[vidId] = {'validityRatio': len(validFrameList)/len(frameList), 'mse_stats': getMeanMedianStdVar(listOfMSE), 'ssim_stats': getMeanMedianStdVar(listOfSIMM)
    }
count = 0
totalFoldersToGoThrough = len(glob.glob("frames/*"))    
for folder in glob.glob("frames/*"):
    count+=1
    print(">", folder, "{}/{}".format(count, totalFoldersToGoThrough))
    
    if folder.split('/')[-1] not in not_candidate_vidIds:
        removeLowDiffDst = removeLowDiff_andDemo(folder)

In [ ]:
# with open('vidIdToFramesListAndStats94.json', 'w') as output:
#     json.dump(dictOfVidIdToDiffMse, output)
with open('vidIdToFramesListAndStats94.json') as input:
    data = json.load(input)
dictOfVidIdToDiffMse = data

In [ ]:
%matplotlib widget

In [ ]:
#mse mean plot
mse_mean=[dictOfVidIdToDiffMse[vidId]['mse_stats']['mean'] for vidId in dictOfVidIdToDiffMse.keys()]
print("min mse_mean:", min(mse_mean))
print("max mse_mean:", max(mse_mean))
binAndPlot(mse_mean)

In [ ]:
vidIdToFramesListAndStatsDf = pd.DataFrame.from_dict(data).T

with open('../channelId_to_mostViewedVidId.json') as input:
    channelId_to_mostViewedVidId = json.load(input)
with open('../channelHolisticInfo.json') as input:
    channelHolisticInfo = pd.DataFrame.from_dict(json.load(input))

channelIdInOrder = []
for index in vidIdToFramesListAndStatsDf.index:
    for channelId, mostViewedVidId in channelId_to_mostViewedVidId.items():
        if index == mostViewedVidId:
            channelIdInOrder.append(channelId)

channelTitle=[]
channelSubCountNormalized=[]
for channelId in channelIdInOrder:
    for index, row in channelHolisticInfo.iterrows():
        if row['channelId'] == channelId:
            channelTitle.append(row['channelName'])
            channelSubCountNormalized.append(row['subscribersPerYearSinceFirstUpload'])
channelHolisticInfo.head()
vidIdToFramesListAndStatsDf['channelId'] = channelIdInOrder
vidIdToFramesListAndStatsDf['channelName'] = channelTitle
vidIdToFramesListAndStatsDf['subscribersPerYearSinceFirstUpload'] = channelSubCountNormalized

colorRatioInOrder = []
for index in vidIdToFramesListAndStatsDf.index:
    for video, colorRatio in vidId_to_colorPercentage_2.items():
        if index == video:
            colorRatioInOrder.append(colorRatio)

vidIdToFramesListAndStatsDf['colorRatio'] = colorRatioInOrder


In [ ]:
vidIdToFramesListAndStatsDf

In [ ]:
%matplotlib widget

In [ ]:
import mplcursors
import scipy

x_name = "subscribersPerYearSinceFirstUpload"
y_name = "colorRatio"
tooltip_name = "channelName"
x = vidIdToFramesListAndStatsDf[x_name]
y= vidIdToFramesListAndStatsDf[y_name]

tt = vidIdToFramesListAndStatsDf[tooltip_name]

zipped = zip(x, y)
sorted_vidId_to_validRatio = dict(sorted(dict(zipped).items(), key=lambda item: item[1]))

fig, ax = plt.subplots(1, figsize=(12,6))
sc = ax.scatter(sorted_vidId_to_validRatio.keys(), sorted_vidId_to_validRatio.values())
# # Plot the average line
# avg_score= vidIdToFramesListAndStatsDf[y_name].mean()
# mean_x=np.arange(0,43,1)
# mean_y=[avg_score for i in np.zeros_like(x)]
# mean_line = ax.plot(mean_x,mean_y, label='Mean', linestyle='--')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
cursor = mplcursors.cursor(sc, hover=True)
cursor.connect(
"add", lambda sel: sel.annotation.set_text(tt[sel.index]))
plt.xlabel(x_name)
plt.ylabel(y_name)

result = scipy.stats.linregress(x, y)
m=result.slope
b=result.intercept
plt.plot([i for i in x], [m*i + b for i in x])
plt.show()

print("rvalue: ", result.rvalue)

In [ ]:
import mplcursors
import scipy

x_name = "validityRatio"
y_name = "subscribersPerYearSinceFirstUpload"
tooltip_name = "channelName"
x = [float(i) for i in vidIdToFramesListAndStatsDf[x_name]]
y= [float(i) for i in vidIdToFramesListAndStatsDf[y_name]]

tt = vidIdToFramesListAndStatsDf[tooltip_name]

zipped = zip(x, y)
sorted_vidId_to_validRatio = dict(sorted(dict(zipped).items(), key=lambda item: item[1]))
# tt = df[tooltip_name]

fig, ax = plt.subplots(1, figsize=(12,6))
sc = ax.scatter(x,y)
# # Plot the average line
# avg_score= vidIdToFramesListAndStatsDf[y_name].mean()
# mean_x=np.arange(0,43,1)
# mean_y=[avg_score for i in np.zeros_like(x)]
# mean_line = ax.plot(mean_x,mean_y, label='Mean', linestyle='--')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
cursor = mplcursors.cursor(sc, hover=True)
cursor.connect(
"add", lambda sel: sel.annotation.set_text(tt[sel.index]))
plt.xlabel(x_name)
plt.ylabel(y_name)

result = scipy.stats.linregress(x, y)
m=result.slope
b=result.intercept
plt.plot([i for i in x], [m*i + b for i in x])
plt.show()

print("rvalue: ", result.rvalue)

In [ ]:
import scipy
import numpy as np

x= 'validityRatio'	
y= 'subscribersPerYearSinceFirstUpload'
result = scipy.stats.linregress([float(i) for i in vidIdToFramesListAndStatsDf[x]], [float(i) for i in vidIdToFramesListAndStatsDf[y]])
color = scipy.stats.linregress([float(i) for i in vidIdToFramesListAndStatsDf['colorRatio']], [float(i) for i in vidIdToFramesListAndStatsDf[y]])
ssim_mean = scipy.stats.linregress([i['mean'] for i in vidIdToFramesListAndStatsDf['ssim_stats']], vidIdToFramesListAndStatsDf[y])
mse_mean = scipy.stats.linregress([i['mean'] for i in vidIdToFramesListAndStatsDf['mse_stats']], vidIdToFramesListAndStatsDf[y])
ssim_std = scipy.stats.linregress([i['std'] for i in vidIdToFramesListAndStatsDf['ssim_stats']], vidIdToFramesListAndStatsDf[y])
mse_std = scipy.stats.linregress([i['std'] for i in vidIdToFramesListAndStatsDf['mse_stats']], vidIdToFramesListAndStatsDf[y])

print("validityRatio: ", 'rvalue:{}, pvalue:{}'.format(result.rvalue, result.pvalue))
print("colorRatio: ", 'rvalue:{}, pvalue:{}'.format(color.rvalue, color.pvalue))
print("ssim_mean: ", 'rvalue:{}, pvalue:{}'.format(ssim_mean.rvalue, ssim_mean.pvalue))
print("mse_mean: ", 'rvalue:{}, pvalue:{}'.format(mse_mean.rvalue, mse_mean.pvalue))
print("ssim_std: ", 'rvalue:{}, pvalue:{}'.format(ssim_std.rvalue, ssim_std.pvalue))
print("mse_std: ", 'rvalue:{}, pvalue:{}'.format(mse_std.rvalue, mse_std.pvalue))
print()

In [ ]:
def getSSIMBetweenFrames(folderName):
    vidId = folderName.split('/')[-1]
    frameList = glob.glob(folderName+'/*.png') #unfortunately, doesn't work bc sortByInt isn't a thing

    listOfSIMM=[]

    for i in range(len(frameList)-1):
        firstFrame = '{}/{}.png'.format(folderName, str(i).zfill(5))
        secondFrame = '{}/{}.png'.format(folderName, str(i+1).zfill(5))
        score = SSIM_images(firstFrame, secondFrame, listOfSIMM)
        print("{} || SSIM score between {}.png and {}.png: {}".format(vidId, str(i).zfill(5), str(i+1).zfill(5), score ))

# totalFoldersToGoThrough = len(glob.glob("frames/*"))    
# for folder in glob.glob("frames/*"):
#     if folder.split('/')[-1] not in not_candidate_vidIds:
#         removeLowDiffDst = getSSIMBetweenFrames(folder)
image = cv2.imread('download.jpeg')
resized = cv2.resize(image, (256,256), interpolation = cv2.INTER_AREA)
cv2.imwrite( '1.jpeg', resized)
image2 = cv2.imread('unnamed.webp')
resized2 = cv2.resize(image2, (256,256), interpolation = cv2.INTER_AREA)
cv2.imwrite('2.jpeg', resized2)

p = SSIM_images('1.jpeg', '2.jpeg', [])
p